<a href="https://colab.research.google.com/github/ettore9x9/machine_learning_examples/blob/main/ex6%2B_minst_keras_gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Nov 25 20:16:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
from tensorflow import keras
from keras import layers

In [3]:
c = 10
s = (28,28,1)

(xl,yl),(xt,yt) = keras.datasets.mnist.load_data()

In [4]:
xl = xl.astype("float32") / 255.0
xt = xt.astype("float32") / 255.0

In [5]:
np.shape(xl)

(60000, 28, 28)

In [6]:
xl = np.expand_dims(xl,-1)
xt = np.expand_dims(xt,-1)

In [7]:
np.shape(yl)

(60000,)

In [8]:
yl = keras.utils.to_categorical(yl,c)
yt = keras.utils.to_categorical(yt,c)

In [9]:
np.shape(yl)

(60000, 10)

In [10]:
model = keras.Sequential(
    [
        keras.Input(s),
        layers.Conv2D(32,kernel_size=(3,3),activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Conv2D(64,kernel_size=(3,3),activation="relu"),
        layers.MaxPooling2D(pool_size=(2,2)),
        layers.Flatten(),
        layers.Dropout(.5),
        layers.Dense(c,activation="softmax"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dropout (Dropout)           (None, 1600)              0

In [11]:
from tensorflow.python import metrics
epochs = 15
# stochastic gradient descent, to avoid memorising but instead learn
batch_size = 128
model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.fit(xl,
          yl,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=.1)

Epoch 1/15
422/422 [==============================] - 8s 10ms/step - loss: 0.3761 - accuracy: 0.8841 - val_loss: 0.0904 - val_accuracy: 0.9763
Epoch 2/15
422/422 [==============================] - 4s 10ms/step - loss: 0.1147 - accuracy: 0.9648 - val_loss: 0.0569 - val_accuracy: 0.9848
Epoch 3/15
422/422 [==============================] - 2s 6ms/step - loss: 0.0860 - accuracy: 0.9733 - val_loss: 0.0528 - val_accuracy: 0.9847
Epoch 4/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0729 - accuracy: 0.9773 - val_loss: 0.0415 - val_accuracy: 0.9888
Epoch 5/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0634 - accuracy: 0.9805 - val_loss: 0.0387 - val_accuracy: 0.9898
Epoch 6/15
422/422 [==============================] - 2s 6ms/step - loss: 0.0569 - accuracy: 0.9825 - val_loss: 0.0374 - val_accuracy: 0.9885
Epoch 7/15
422/422 [==============================] - 2s 5ms/step - loss: 0.0527 - accuracy: 0.9837 - val_loss: 0.0338 - val_accuracy: 0.9910
Epoc

In [12]:
acc = model.evaluate(xt,yt)
print(acc[1])

313/313 [==============================] - 1s 3ms/step - loss: 0.0250 - accuracy: 0.9924
0.9923999905586243


In [13]:
from sklearn.metrics import confusion_matrix
yp=model.predict(xt)
yp=yp.argmax(1)
yt=yt.argmax(1)
confusion_matrix(yt,yp)

313/313 [==============================] - 1s 2ms/step


array([[ 974,    0,    3,    1,    0,    0,    2,    0,    0,    0],
       [   0, 1131,    3,    0,    0,    0,    1,    0,    0,    0],
       [   0,    0, 1031,    0,    0,    0,    0,    1,    0,    0],
       [   0,    0,    3, 1006,    0,    1,    0,    0,    0,    0],
       [   0,    0,    1,    0,  975,    0,    4,    0,    0,    2],
       [   1,    0,    0,    6,    0,  884,    1,    0,    0,    0],
       [   1,    2,    1,    0,    1,    3,  950,    0,    0,    0],
       [   0,    2,    6,    1,    0,    1,    0, 1016,    1,    1],
       [   1,    0,    6,    1,    1,    1,    0,    1,  960,    3],
       [   0,    0,    0,    2,    3,    3,    0,    3,    1,  997]])

In [16]:
%%capture
!pip install gradio

In [19]:
import gradio as gr
def classify(input):
    prediction = model.predict(input.reshape(1, 28, 28)).tolist()[0]
    return {str(i): prediction[i] for i in range(10)}

label = gr.outputs.Label(num_top_classes=3)
interface = gr.Interface(fn=classify, inputs="sketchpad", outputs=label, live=False)

/usr/local/lib/python3.7/dist-packages/gradio/outputs.py:197: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:44: UserWarning: You have unused kwarg parameters in Sketchpad, please remove them: {'visibile': True}
  f"You have unused kwarg parameters in {cls}, please remove them: {kwargs}"


In [20]:
interface.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>